# COVID-19 Interactive Analysis Dashboard

## What is COVID-19?
Coronaviruses are a large family of viruses that may cause respiratory illnesses in humans ranging from common colds to more severe conditions such as Severe Acute Respiratory Syndrome (SARS) and Middle Eastern Respiratory Syndrome (MERS).1
'Novel coronavirus' is a new, previously unidentified strain of coronavirus. The novel coronavirus involved in the current outbreak has been named SARS-CoV-2 by the World Health Organization (WHO). 3The disease it causes has been named “coronavirus disease 2019” (or “COVID-19”).`

![Coronavirus particle Image](img.jpg)

In [1]:
#importing packages
from __future__ import print_function
import numpy as np
import pandas as pd


from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
# loading data right from the source:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [3]:
#death_df.head()

In [4]:
#confirmed_df.head()

In [5]:
#confirmed_df.isnull().sum()

In [6]:
confirmed_df.dropna(subset=['Lat'],inplace=True)
#confirmed_df.isnull().sum()

In [38]:
#recovered_df.head()

In [8]:
#country_df.head()

In [9]:
#data cleaning-renaming columns
country_df.columns=map(str.lower,country_df.columns)
recovered_df.columns=map(str.lower,recovered_df.columns)
confirmed_df.columns=map(str.lower,confirmed_df.columns)
death_df.columns=map(str.lower,death_df.columns)

In [10]:
country_df=country_df.drop(['incident_rate','people_tested','people_hospitalized', 'mortality_rate', 'uid',
       'iso3'],axis=1)
#country_df.columns

In [11]:
#country_df.head()

In [12]:
confirmed_df=confirmed_df.rename(columns={'province/state':'state', 'country/region':'country'})
death_df=death_df.rename(columns={'province/state':'state', 'country/region':'country'})
recovered_df=recovered_df.rename(columns={'province/state':'state', 'country/region':'country'})
country_df=country_df.rename(columns={'country_region':'country'})

In [13]:
sorted_country_df=country_df.sort_values('confirmed',ascending=False)

In [49]:
sorted_country_df.iloc[0,6]=29435171
sorted_country_df.iloc[0,7]=4130074
#sorted_country_df.head()

,country,last_update,lat,long_,confirmed,deaths,recovered,active
181,US,2021-07-21 06:21:15,40.000000,-100.00000,34174774.0,609529.0,29435171.0,4130074.0
79,India,2021-07-21 06:21:15,20.593684,78.96288,31216337.0,418480.0,30390687.0,407170.0


In [15]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())



## Total Cases:



In [24]:
print("Confirmed: " + str(confirmed_total))
print("Deaths: " + str(deaths_total))
print("Revovered: " + str(recovered_total))

Confirmed: 191451690
Deaths: 4118473
Revovered: 126009007


# COVID-19 Confirmed/Death/Recovered cases by countries

## Enter number of countries you want the data for

In [16]:
#we define a highlight function to highlight the columns to get a better view

def highlight_col(x):
    r='background-color: red'
    p='background-color: purple'
    g='background-color: grey'
    temp_df=pd.DataFrame('',index=x.index,columns=x.columns)
    temp_df.iloc[:,4]=p
    temp_df.iloc[:,5]=r
    temp_df.iloc[:,6]=g
    return temp_df

#sorted_country_df.style.apply(highlight_col,axis=None)


def show_cases(n):
    n=int(n)
    return sorted_country_df.head(n).style.apply(highlight_col,axis=None)
    
interact(show_cases, n='10')
    
ipywLayout = widgets.Layout(border='solid 2px green')

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))







# Slide to check for the worst hit countries

In [17]:
import plotly.express as px

In [18]:
def plot(n=10):
    fig=px.scatter(sorted_country_df.head(n),x='country',y='confirmed',size='confirmed',color='country',hover_name='country',size_max=50
                  ,title=str(n)+"Worst Hit Countries")
    fig.show()
    
interact(plot,n=(1,20,1))
ipywLayout = widgets.Layout(border='solid 2px green')

interactive(children=(IntSlider(value=10, description='n', max=20, min=1), Output()), _dom_classes=('widget-in…

# Check the details of your country or the World

* Enter the name of your country(in capitalized format(e.g. Italy)) and world for total cases

In [19]:
import plotly.graph_objects as go

def plot_cases_for_country(country):
    labels=['confirmed','deaths']
    colors=['blue','red']
    mode_size= [6,8]
    line_size=[4,5]

    df_list=[confirmed_df,death_df]
    fig=go.Figure()

    for i,df in enumerate(df_list):
        if country=='World' or country=='world':
            x_data=np.array(list(df.iloc[:,5:].columns))
            y_data=np.sum(np.asarray(df.iloc[:,5:]),axis=0)
        
        else:
            x_data=np.array(list(df.iloc[:,5:].columns))
            y_data=np.sum(np.asarray(df[df['country']==country].iloc[:,5:]),axis=0);
        
        fig.add_trace(go.Scatter(x=x_data,y=y_data,mode='lines+markers',
                                 name=labels[i],
                                 line=dict(color=colors[i], width=line_size[i]),
                                 connectgaps=True,
                                 text="Total "+str(labels[i])+": "+str(y_data[-1])
                                 ))
    fig.show()

#plot_cases_for_country('India')

interact(plot_cases_for_country, country='World');

interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

# Global spread of COVID-19

In [20]:
import folium

In [21]:

world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)


for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*20000,
        color='red',
        fill_color='indigo',
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map
